In [1]:
# Задание:

# По представленным данным определить следующую наиболее вероятную дату совершения повторной покупки по каждому клиенту
# По факту выполнения первого пункта разбейте клиентов на группы:
# - лояльные / активные
# - склонные к оттоку
# - в оттоке

# Если вы считаете, что групп должно быть больше, то выделите столько, сколько считаете правильным, и объясните логику.
# Представьте, что это данные b2b-клиентов. 
# Перечислите список факторов, которые вы бы использовали для построения модели оттока, 
# если у вас есть возможность получить любую информацию о клиентах и их действиях?
# Результатом работы должен быть работающий код для Jupiter Notebook и выводы по итогам расчетов.

In [1]:

from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

from django.utils import dateparse
from datetime import timedelta
import datetime


path = '/Volumes/Data/ML/rabotaru/'
os.chdir(path)

In [2]:

def got_counter_res_df_speed1(db):
    res_df = pd.DataFrame(columns = ['event', 'count', '%'])
    if len(db) > 0:
        unique, counts = np.unique(db, return_counts=True); 
        counter = dict(zip(unique, counts))
        res_df = pd.DataFrame.from_dict(counter, orient='index').reset_index().rename(columns={'index':'event', 0:'count'})
        res_df = res_df.sort_values(by=['count'], ascending=False).reset_index(drop=True)
        res_df['%'] = 0.0;
        for i in (range(len(res_df))): res_df.loc[i, '%'] = int((res_df.loc[i, 'count']*100/(np.sum(res_df['count'])))*1000)/1000    
    return res_df

def fun03(x):
    vc0 = [];
    vc1 = []
    for M in tqdm([x for x in range(10, 100)]):
        n = int(len(x)/M)
        rs = []
        for i in [x for x in range(0, M+2)]:
            if i*n < len(x):
                y = x[i*n:i*n+n].copy()
                if i == M+ 1:
                    y = x[i*n:].copy()
                rs.append([np.min(y), np.max(y), np.mean(y), np.median(y), np.std(y), np.abs(np.mean(y) - np.median(y))])
        rs = pd.DataFrame(rs, columns = ['min', 'max', 'mean', 'median', 'std', 'delta'])
        rs2 = rs[rs['delta'] > np.median(rs['delta']) + np.std(rs['delta'])*3].reset_index(drop=True)
        if len(rs2) > 0:
            vc0.append(rs2.iloc[-1, 0])
            vc1.append(rs2.iloc[-1, 1])
    a = np.mean(vc0)
    return a

def calc_N_minutes_2(dbs, A, verbose):
    def calcs(s, A):
        dr = dateparse.parse_datetime(s)
        B = str(int(dr.minute/A)*A)
        if len(B) == 1: B = '0'+B
        e = dateparse.parse_datetime(s).strftime('%Y-%m-%d %H:'+B+':00')
        return e
    if verbose:
        dbs = [calcs(x, A) for x in tqdm(dbs)]
    else:
        dbs = [calcs(x, A) for x in (dbs)]
    return dbs

def transform_datetime_features_3(df, col_name, A, time_in_sec):
    # col_name = 'REPORT_DATE'
    # df = ds.copy()
    def parse_dt(x):
        x = str(x)
        x = x[:19]
        y = datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
        return y
    
    df['year'] = df[col_name].apply(lambda x: parse_dt(x).year)
    #years = list(df['year'].unique())
    #num_of_years = len(years)
    
    df['month'] = df[col_name].apply(lambda x: parse_dt(x).month)
    df['day'] = df[col_name].apply(lambda x: parse_dt(x).day)
    df['hour'] = df[col_name].apply(lambda x: parse_dt(x).hour)
    df['minute'] = df[col_name].apply(lambda x: parse_dt(x).minute)
    df['second'] = df[col_name].apply(lambda x: parse_dt(x).second)
    df['weekday'] = df[col_name].apply(lambda x: parse_dt(x).weekday()) 

       
    df['day_of_year'] =  [int(str(dateparse.parse_datetime(dt).strftime('%j'))) for dt in df[col_name]]
    df['week_of_year'] = [int(str(dateparse.parse_datetime(dt).strftime('%V'))) for dt in df[col_name]]
    
    df['week_of_years'] = [ str(parse_dt(dt).year)+'_'+str(dateparse.parse_datetime(dt).strftime('%V')) for dt in df[col_name]]
    
    df[col_name]  = calc_N_minutes_2(list(df[col_name]), A, verbose=False)    
    df['num_of_day'] =  [int((parse_dt(dt).hour*60+parse_dt(dt).minute)/A) for dt in df[col_name]]
    df['num_of_week'] =  [int((parse_dt(dt).hour*60+parse_dt(dt).minute)/A)+int(3600*24/time_in_sec/A)*parse_dt(dt).weekday() for dt in df[col_name]]
    df['num_of_hour'] =  [str(parse_dt(dt).weekday())+'_'+str(parse_dt(dt).hour) for dt in df[col_name]]
    
    df['weekday_hour'] =  [str(parse_dt(dt).weekday())+'_'+str(parse_dt(dt).hour) for dt in df[col_name]]
    
    df['month_weekday_hour'] =  [str(parse_dt(dt).month)+'_'+str(parse_dt(dt).weekday())+'_'+str(parse_dt(dt).hour) for dt in df[col_name]]
    
    df['month_week'] =  [ str(parse_dt(dt).year)+'_'+str(dateparse.parse_datetime(dt).strftime('%V')) for dt in df[col_name]]
    
    df['year_month_weekday_hour'] =  [str(parse_dt(dt).year)+'_'+str(parse_dt(dt).month)+'_'+str(parse_dt(dt).weekday())+'_'+str(parse_dt(dt).hour) for dt in df[col_name]]    
    
    df['weekinmonth_weekday_hour'] =  [str(parse_dt(dt).day // 7)+'_'+str(parse_dt(dt).weekday())+'_'+str(parse_dt(dt).hour) for dt in df[col_name]]
    
    df['year_month'] =  [str(parse_dt(dt).year)+'_'+str(parse_dt(dt).month) for dt in df[col_name]]
    df['year_month_day'] =  [str(parse_dt(dt).year)+'_'+str(parse_dt(dt).month)+'_'+str(parse_dt(dt).day)  for dt in df[col_name]]
    
    #df['week_in_month'] = df[col_name].apply(lambda x: parse_dt(x).day // 7)
    
    return df



def day_of_week(int_val):
    remainder = int_val % 7
    if (remainder == 0):
        return 7
    else:
        return remainder
	  
		  
def week_by_day_origin_sigma2(num_day,sigma2):
    return (1+(num_day-day_of_week(num_day))/7)**sigma2

def overall_week_score_sigma2(sigma2):
    res =0
    number_of_weeks = 99
    for i in range(1,int(number_of_weeks+1)):
        res+=i**sigma2
    return res

def res_day_of_week_sigma3(value_visits, sigma2 = 1.1):
    individual_p1=0
    individual_p2=0
    individual_p3=0
    individual_p4=0
    individual_p5=0
    individual_p6=0
    individual_p7=0
    for value_visit in value_visits:
        day_of_visit = int(value_visit)
        if (day_of_week(day_of_visit) == 1): individual_p1 += 1*week_by_day_origin_sigma2(day_of_visit,sigma2)
        if (day_of_week(day_of_visit) == 2): individual_p2 += 1*week_by_day_origin_sigma2(day_of_visit,sigma2)
        if (day_of_week(day_of_visit) == 3): individual_p3 += 1*week_by_day_origin_sigma2(day_of_visit,sigma2)
        if (day_of_week(day_of_visit) == 4): individual_p4 += 1*week_by_day_origin_sigma2(day_of_visit,sigma2)
        if (day_of_week(day_of_visit) == 5): individual_p5 += 1*week_by_day_origin_sigma2(day_of_visit,sigma2)
        if (day_of_week(day_of_visit) == 6): individual_p6 += 1*week_by_day_origin_sigma2(day_of_visit,sigma2)
        if (day_of_week(day_of_visit) == 7): individual_p7 += 1*week_by_day_origin_sigma2(day_of_visit,sigma2)
    individual_p1 = float(individual_p1)/overall_week_score_sigma2(sigma2)
    individual_p2 = float(individual_p2)/overall_week_score_sigma2(sigma2)
    individual_p3 = float(individual_p3)/overall_week_score_sigma2(sigma2)
    individual_p4 = float(individual_p4)/overall_week_score_sigma2(sigma2)
    individual_p5 = float(individual_p5)/overall_week_score_sigma2(sigma2)
    individual_p6 = float(individual_p6)/overall_week_score_sigma2(sigma2)
    individual_p7 = float(individual_p7)/overall_week_score_sigma2(sigma2)
    p1 = individual_p1
    p2 = (1-p1)*individual_p2
    p3 = (1-p1)*(1-p2)*individual_p3
    p4 = (1-p1)*(1-p2)*(1-p3)*individual_p4
    p5 = (1-p1)*(1-p2)*(1-p3)*(1-p4)*individual_p5
    p6 = (1-p1)*(1-p2)*(1-p3)*(1-p4)*(1-p5)*individual_p6
    p7 = (1-p1)*(1-p2)*(1-p3)*(1-p4)*(1-p5)*(1-p6)*individual_p7
    probs = [p1,p2,p3,p4,p5,p6,p7]
    iprobs = np.sum([individual_p1,individual_p2,individual_p3,individual_p4,individual_p5,individual_p6,individual_p7])
    max_prob = max(probs)
    res_day_of_week = 0
    for i in range(len(probs)):
        if (probs[i] == max_prob):
            res_day_of_week = i+1
            break
    return res_day_of_week, max_prob, iprobs


def calc_repdate_4(repdate, time_in_sec = 600):
    
    repd = [x for x in repdate]
    for i in range(len(repd)):
        if len(repd) == 10:
            repd[i] = repd[i] + ' 00:00:00'
    
    startdate = repd[0]
    stopdate = repd[-1]
    repdate2 = [startdate]
    if (dateparse.parse_datetime(stopdate) > dateparse.parse_datetime(startdate)):
        d = (dateparse.parse_datetime(stopdate) - dateparse.parse_datetime(startdate)).days
        d = int(d*24*3600/time_in_sec)
        n = int((dateparse.parse_datetime(stopdate) - dateparse.parse_datetime(startdate)).seconds/time_in_sec)
        tmpdate = startdate
        for i in range(d+n):
            tmpdate = (dateparse.parse_datetime(tmpdate)+timedelta(seconds=time_in_sec)).strftime('%Y-%m-%d %H:%M:%S')
            if (dateparse.parse_datetime(tmpdate) <= dateparse.parse_datetime(stopdate)):
                repdate2.append(tmpdate)
    repdate2.sort()
    return repdate2


def parse_dt(x):
    x = str(x)
    x = x[:19]
    y = datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
    return y



def calc_N_bitween_dates_equill_years(datestart, datestop, time_in_sec):
    N = 0
    if (dateparse.parse_datetime(datestop) >= dateparse.parse_datetime(datestart)):
        d = (dateparse.parse_datetime(datestop) - dateparse.parse_datetime(datestart)).days
        d = int(d*24*3600/time_in_sec)
        n = int((dateparse.parse_datetime(datestop) - dateparse.parse_datetime(datestart)).seconds/time_in_sec)
        N = d+n
    else:
        d = (dateparse.parse_datetime(datestart) - dateparse.parse_datetime(datestop)).days
        d = int(d*24*3600/time_in_sec)
        n = int((dateparse.parse_datetime(datestart) - dateparse.parse_datetime(datestop)).seconds/time_in_sec)
        N = d+n    
    return N

def convdt(f):
    f = f.split('_')
    yy = f[0]  
    if len(f[1]) < 2:
        mm = '0'+f[1]
    else:
        mm = f[1]
    if len(f[2]) < 2:
        dd = '0'+f[2]
    else:
        dd = f[2]
    ymd = yy+'-'+mm+'-'+dd+' 00:00:00'
    return ymd

def calc_diff_time(value_visits):
    time_in_sec = 60
    s = list(value_visits[value_visits > 0].index)
    sn = []
    for i in range(len(s)):
        f = s[i]
        ymd = convdt(f)
        sn.append(ymd)
    sn.sort()    
    
    v = []
    for i in range(len(sn)-1):
        datestart = sn[i]
        datestop =  sn[i+1]
        v.append(int(calc_N_bitween_dates_equill_years(datestart, datestop, time_in_sec)/60/24))
    return v

### Загрузка данных

In [4]:

input_filename = 'file.csv'
input_filename = 'train.csv'
ds = pd.DataFrame()
ds = pd.read_csv(os.path.join(os.getcwd(), input_filename), sep=';', encoding='cp1251')
ds.fillna('', inplace=True)
ds = ds.drop_duplicates().reset_index(drop=True)
if 'Unnamed: 0' in ds.columns: del ds['Unnamed: 0']

In [5]:
ds.head()

,"id,visits"
0,"1, 30 84 126 135 137 179 242 342 426 456 460 4..."
1,"2, 24 53 75 134 158 192 194 211 213 238 251 30..."
2,"3, 51 143 173 257 446 491 504 510 559 616 719 ..."
3,"4, 1 20 22 92 124 149 211 335 387 390 406 460 ..."
4,"5, 34 51 56 106 110 121 163 233 266 275 345 35..."


### Добавление признака: количество покупок клиента

In [6]:
'''
c = 'ID КЛИЕНТА'
res_df = got_counter_res_df_speed1(ds[c])
res_df = res_df[['event', 'count']]
res_df = res_df.rename(columns={'event':'ID КЛИЕНТА'})
ds = ds.merge(res_df, how = 'left', left_on=['ID КЛИЕНТА'] , right_on=['ID КЛИЕНТА'])

res_df = got_counter_res_df_speed1(ds['count'])
res_df.columns = ['Количество покупок', 'Сколько раз встречается', '% от общего числа']
'''

"\nc = 'ID КЛИЕНТА'\nres_df = got_counter_res_df_speed1(ds[c])\nres_df = res_df[['event', 'count']]\nres_df = res_df.rename(columns={'event':'ID КЛИЕНТА'})\nds = ds.merge(res_df, how = 'left', left_on=['ID КЛИЕНТА'] , right_on=['ID КЛИЕНТА'])\n\nres_df = got_counter_res_df_speed1(ds['count'])\nres_df.columns = ['Количество покупок', 'Сколько раз встречается', '% от общего числа']\n"

In [7]:
#res_df.head()

In [8]:
"""
# оставим только тех клиентов, по которым есть историческая информация (Больше одной покупки, иначе предсказать затруднительно)
lends1 = len(ds[ds['count'] < 2])
ds = ds[ds['count'] >= 2].reset_index(drop=True)
lends2 = len(ds)
print('Отбросим', str(lends1), 'строк с неактивными клиентами')
print('Анализируем', str(lends2), 'строк с активными клиентами')
"""

"\n# оставим только тех клиентов, по которым есть историческая информация (Больше одной покупки, иначе предсказать затруднительно)\nlends1 = len(ds[ds['count'] < 2])\nds = ds[ds['count'] >= 2].reset_index(drop=True)\nlends2 = len(ds)\nprint('Отбросим', str(lends1), 'строк с неактивными клиентами')\nprint('Анализируем', str(lends2), 'строк с активными клиентами')\n"

In [8]:
#Посмотрим какое распределение количества покупок в день у каждого клиента
c1 = 'ДАТА ЗАКАЗА'
uniq = list(ds[c].unique())
v = []
for i in (range(len(uniq))):
    d = ds[ds[c] == uniq[i]].reset_index(drop=True)
    z = len(d) - len(d[c1].unique())
    v.append(z)

res_df = got_counter_res_df_speed1(v)
res_df.columns = ['Количество покупок', 'Сколько раз встречается', '% от общего числа']
# 90% 1 per day


In [9]:
# 90%  - 1 покупка в день
res_df.head()

,Количество покупок,Сколько раз встречается,% от общего числа
0,0,6731,90.022
1,1,596,7.971
2,2,84,1.123
3,3,25,0.334
4,4,12,0.160


### Создать матрицу timeseries

In [10]:
# Преобразовать дату в правильный формат
ds[c1] = [x.split('.')[2]+'-'+x.split('.')[1]+'-'+x.split('.')[0]+' 00:00:00' for x in ds[c1]]
# Добавить признак уникальной даты
ds['year_month_day'] =  [str(parse_dt(dt).year)+'_'+str(parse_dt(dt).month)+'_'+str(parse_dt(dt).day)  for dt in ds[c1]]
# Добавить признак - день недели
ds['weekday'] = ds[c1].apply(lambda x: parse_dt(x).weekday() + 1)

###   В работе используются 2 метода прогнозирования:
1. Расчет наиболее вероятного дня недели совершения следующего визита
2. Расчет совершения следующего дня визита

Первый метод улучшает решение второго


In [11]:
repdate = ['2019-04-29 00:00:00', '2021-03-21 00:00:00']
repdate = calc_repdate_4(repdate, time_in_sec = 3600*24)
num_of_week = int(len(repdate)/7)
repdate_c =  [str(parse_dt(dt).year)+'_'+str(parse_dt(dt).month)+'_'+str(parse_dt(dt).day)  for dt in repdate]

In [12]:
uniq = list(ds[c].unique())
v = []
for i in tqdm(range(len(uniq))):
    d = ds[ds[c] == uniq[i]].reset_index(drop=True)
    ls = list(d['year_month_day'])
    z = [uniq[i]] + [1 if x in ls else 0 for x in repdate_c]
    v.append(z)
megamatr = pd.DataFrame(v, columns = ['ID']+repdate_c)

100%|██████████| 7477/7477 [00:04<00:00, 1811.05it/s]


In [13]:
# Получили сводную матрицу, где в каждой строке указан ID клиента и единицей показан день, когда был совершен визит, 
# который закодирован в имени столбца
megamatr.head()

,ID,2019_4_29,2019_4_30,2019_5_1,2019_5_2,2019_5_3,2019_5_4,2019_5_5,2019_5_6,2019_5_7,...,2021_3_12,2021_3_13,2021_3_14,2021_3_15,2021_3_16,2021_3_17,2021_3_18,2021_3_19,2021_3_20,2021_3_21
0,1632,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
1,124509,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,43791,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,85655,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,26081,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [14]:
# Основная функция

def calc_sol_1(megamatr, ndays, A, sigma2 = 1.1):
    # ndays - список [7, 14, 21, ...] сколько дней отбрасываем для тестирования 
    # A - Количество последних визитов для определения следующего дня визита
    # sigma2 - весовой коэффициент
    
    # Вычислим дату начала и когда исследуемого периода
    datestart = convdt(megamatr.columns[1])
    datestop = convdt(megamatr.columns[-1])
    # Список дат, которые отбрасываем, для проверки качества
    sn = [(dateparse.parse_datetime(datestop)-timedelta(days=ndays-x-1)).strftime('%Y-%m-%d %H:%M:%S') for x in range(ndays)]
    N = megamatr.shape[1] - 1
    # Список количества дней, которые отбрасываем, для проверки качества
    sm = [x for x in range(N-ndays, N)]
    v = []; a = [];  b = []
    h = ndays//7
    # Для каждого клиента в отдельности будем предсказывать дату следующего визита и корректировать ее
    # днем недели следующего визита
    for i in tqdm(range(len(megamatr))):
        idx = str(megamatr.iloc[i, 0])
        # Разделю выборку на обучающую (value_visits) и проверочную (value_target)
        value_visits = megamatr.iloc[i, 1:]
        # Список количества дней между визитами
        z = calc_diff_time(value_visits[:-ndays])
        value_visits = value_visits.reset_index(drop=True)
        value_visits = [x if value_visits[x] > 0 else 0 for x in range(len(value_visits)) ]
        value_target = value_visits[-ndays:]
        value_visits = value_visits[:-ndays]
        value_target = [x for x in value_target if x > 0]
        value_visits = [x for x in value_visits if x > 0]
        value_target_weekday = [day_of_week(x) for x in value_target]
        # Вычислю вероятный день недели следующего визита с помощью метода с весовыми коэффициентами
        ans, mprb, iprb = res_day_of_week_sigma3(value_visits, sigma2 = sigma2)
        # Список дней недели в проверяемом периоде
        vx = [ans+x*7-1 for x in range(h)]
        # Предположительные дни следующего визита
        dte = [sn[x] for x in range(len(sn)) if x in vx]
        
        # Вычисляю дату следующего визита
        dts = ''
        tar_vol = 0
        if len(z) > 1:
            # Беру А последних дней визитов и вычисляю медиану
            m = int(np.median(z[-A:]))
            # Предполагаю, что следующий визит будет совершен через медианный период послед последнего визита
            tar_vol = value_visits[-1] + m
            # если предполагаем, что в следубщие ndays будет визит
            if tar_vol in sm:
                dts = sn[sm.index(tar_vol)]
                # Уточняю дату следующего визита по дню недели следующего визита
                if len(dte) > 1:
                    mz = [int(calc_N_bitween_dates_equill_years(dte[x], dts, time_in_sec)/60/24) for x in range(len(dte))]
                    dts = dte[mz.index(np.min(mz))]
                else:
                    dts = dte[0]
                b.append(dts)
            else:
                dts = (dateparse.parse_datetime(datestart)+timedelta(days=tar_vol)).strftime('%Y-%m-%d %H:%M:%S')
                b.append(dts)
        if dts == '':
            # Если не определил дату следующего визита, то ставлю конец года 
            dts = '2021-12-31 00:00:00'
        
        # dts - дата следующего визита
        
        
        # проверка на правильность       
        flag = 0
        # угадал, что будет визит
        if tar_vol in sm and value_target != []:
            # - совпал день недели
            if ans in value_target_weekday:
                v.append(1)
                a.append([idx, 1, ans, ','.join([str(x) for x in value_target_weekday]), mprb, iprb, dts])
                flag = 1
            # - день недели не угадал
            if ans not in value_target_weekday:
                v.append(0.5)
                if value_target_weekday == []: value_target_weekday = [0]
                a.append([idx, 0.5, ans, ','.join([str(x) for x in value_target_weekday]), mprb, iprb, dts])           
                flag = 1            
        # угадал, что не будет визит в отброшенном периоде
        if tar_vol == 0 and value_target == []:
            if value_target_weekday == []: value_target_weekday = [0]
            v.append(0.25)
            a.append([idx, 0.25, ans, ','.join([str(x) for x in value_target_weekday]), mprb, iprb, dts])
            flag = 1        
        if flag == 0:
            v.append(0)
            if value_target_weekday == []: value_target_weekday = [0]
            a.append([idx, 0, ans, ','.join([str(x) for x in value_target_weekday]), mprb, iprb, dts])        
    return v, a

In [15]:
time_in_sec = 60
ndays = 7
v, a  = calc_sol_1(megamatr, ndays, A = 15, sigma2 = 1.1)
a = pd.DataFrame(a, columns = ['ID', 'rez', 'wday', 'rwday','mprb', 'iprb', 'pred'])
output = a[['ID', 'pred']]
print('Процент точно угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 1])/len(a)*100))
print('Процент не точно угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 0.5])/len(a)*100))
print('Процент, угаданый, что в следущие ndays не будет визита: {:.2f}'.format(len(a[a['rez'] == 0.25])/len(a)*100))
print('Процент не угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 0])/len(a)*100))

100%|██████████| 7477/7477 [00:21<00:00, 344.97it/s]

Процент точно угаданных дней следующего визита: 0.52
Процент не точно угаданных дней следующего визита: 1.86
Процент, угаданый, что в следущие ndays не будет визита: 31.68
Процент не угаданных дней следующего визита: 65.94


In [16]:
ndays = 14
v, a  = calc_sol_1(megamatr, ndays, A = 15, sigma2 = 1.1)
a = pd.DataFrame(a, columns = ['ID', 'rez', 'wday', 'rwday','mprb', 'iprb', 'pred'])
output = a[['ID', 'pred']]
print('Процент точно угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 1])/len(a)*100))
print('Процент не точно угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 0.5])/len(a)*100))
print('Процент, угаданый, что в следущие ndays не будет визита: {:.2f}'.format(len(a[a['rez'] == 0.25])/len(a)*100))
print('Процент не угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 0])/len(a)*100))

100%|██████████| 7477/7477 [00:21<00:00, 340.36it/s]

Процент точно угаданных дней следующего визита: 1.79
Процент не точно угаданных дней следующего визита: 4.65
Процент, угаданый, что в следущие ndays не будет визита: 30.09
Процент не угаданных дней следующего визита: 63.46


In [17]:
ndays = 21
v, a  = calc_sol_1(megamatr, ndays, A = 15, sigma2 = 1.1)
a = pd.DataFrame(a, columns = ['ID', 'rez', 'wday', 'rwday','mprb', 'iprb', 'pred'])
output = a[['ID', 'pred']]
print('Процент точно угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 1])/len(a)*100))
print('Процент не точно угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 0.5])/len(a)*100))
print('Процент, угаданый, что в следущие ndays не будет визита: {:.2f}'.format(len(a[a['rez'] == 0.25])/len(a)*100))
print('Процент не угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 0])/len(a)*100))

100%|██████████| 7477/7477 [00:21<00:00, 352.82it/s]

Процент точно угаданных дней следующего визита: 1.97
Процент не точно угаданных дней следующего визита: 5.84
Процент, угаданый, что в следущие ndays не будет визита: 29.24
Процент не угаданных дней следующего визита: 62.95


In [18]:
ndays = 28
v, a  = calc_sol_1(megamatr, ndays, A = 15, sigma2 = 1.1)
a = pd.DataFrame(a, columns = ['ID', 'rez', 'wday', 'rwday','mprb', 'iprb', 'pred'])
output = a[['ID', 'pred']]
print('Процент точно угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 1])/len(a)*100))
print('Процент не точно угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 0.5])/len(a)*100))
print('Процент, угаданый, что в следущие ndays не будет визита: {:.2f}'.format(len(a[a['rez'] == 0.25])/len(a)*100))
print('Процент не угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 0])/len(a)*100))

100%|██████████| 7477/7477 [00:21<00:00, 353.00it/s]

Процент точно угаданных дней следующего визита: 2.84
Процент не точно угаданных дней следующего визита: 7.24
Процент, угаданый, что в следущие ndays не будет визита: 28.67
Процент не угаданных дней следующего визита: 61.25


In [19]:
ndays = 35
v, a  = calc_sol_1(megamatr, ndays, A = 15, sigma2 = 1.1)
a = pd.DataFrame(a, columns = ['ID', 'rez', 'wday', 'rwday','mprb', 'iprb', 'pred'])
output = a[['ID', 'pred']]
print('Процент точно угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 1])/len(a)*100))
print('Процент не точно угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 0.5])/len(a)*100))
print('Процент, угаданый, что в следущие ndays не будет визита: {:.2f}'.format(len(a[a['rez'] == 0.25])/len(a)*100))
print('Процент не угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 0])/len(a)*100))

100%|██████████| 7477/7477 [00:21<00:00, 352.56it/s]

Процент точно угаданных дней следующего визита: 3.70
Процент не точно угаданных дней следующего визита: 7.50
Процент, угаданый, что в следущие ndays не будет визита: 27.95
Процент не угаданных дней следующего визита: 60.84


In [20]:
ndays = 42
v, a  = calc_sol_1(megamatr, ndays, A = 15, sigma2 = 1.1)
a = pd.DataFrame(a, columns = ['ID', 'rez', 'wday', 'rwday','mprb', 'iprb', 'pred'])
output = a[['ID', 'pred']]
print('Процент точно угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 1])/len(a)*100))
print('Процент не точно угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 0.5])/len(a)*100))
print('Процент, угаданый, что в следущие ndays не будет визита: {:.2f}'.format(len(a[a['rez'] == 0.25])/len(a)*100))
print('Процент не угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 0])/len(a)*100))

100%|██████████| 7477/7477 [00:21<00:00, 352.92it/s]

Процент точно угаданных дней следующего визита: 4.20
Процент не точно угаданных дней следующего визита: 7.86
Процент, угаданый, что в следущие ndays не будет визита: 27.28
Процент не угаданных дней следующего визита: 60.65


In [21]:
ndays = 49
v, a  = calc_sol_1(megamatr, ndays, A = 15, sigma2 = 1.1)
a = pd.DataFrame(a, columns = ['ID', 'rez', 'wday', 'rwday','mprb', 'iprb', 'pred'])
output = a[['ID', 'pred']]
print('Процент точно угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 1])/len(a)*100))
print('Процент не точно угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 0.5])/len(a)*100))
print('Процент, угаданый, что в следущие ndays не будет визита: {:.2f}'.format(len(a[a['rez'] == 0.25])/len(a)*100))
print('Процент не угаданных дней следующего визита: {:.2f}'.format(len(a[a['rez'] == 0])/len(a)*100))

100%|██████████| 7477/7477 [00:24<00:00, 311.25it/s]

Процент точно угаданных дней следующего визита: 5.04
Процент не точно угаданных дней следующего визита: 8.02
Процент, угаданый, что в следущие ndays не будет визита: 26.66
Процент не угаданных дней следующего визита: 60.28


In [22]:
megamatr['ID']

0         1632
1       124509
2        43791
3        85655
4        26081
         ...  
7472       378
7473       507
7474       392
7475       351
7476        22
Name: ID, Length: 7477, dtype: int64

In [23]:
a['ID'] = a['ID'].apply(int)

In [24]:
db = megamatr.merge(a[['ID', 'pred','rez']], how = 'left', left_on=['ID'] , right_on=['ID'])

In [25]:
db

,ID,2019_4_29,2019_4_30,2019_5_1,2019_5_2,2019_5_3,2019_5_4,2019_5_5,2019_5_6,2019_5_7,...,2021_3_14,2021_3_15,2021_3_16,2021_3_17,2021_3_18,2021_3_19,2021_3_20,2021_3_21,pred,rez
0,1632,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,2021-01-23 00:00:00,0.00
1,124509,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,2021-12-31 00:00:00,0.00
2,43791,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,2020-11-18 00:00:00,0.00
3,85655,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,2021-12-31 00:00:00,0.00
4,26081,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,2021-02-08 00:00:00,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7472,378,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2019-07-12 00:00:00,0.00
7473,507,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2021-12-31 00:00:00,0.25
7474,392,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2021-12-31 00:00:00,0.25
7475,351,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2021-12-31 00:00:00,0.25


In [26]:
last_date = '2021-03-21 00:00:00'
v0 = []
v1 = []
v2 = []
for i in tqdm(range(len(db))):
    vv = db.iloc[i, 1:-2]
    last_visit_date = convdt(vv[vv > 0].index[-1])
    days_after_last_visit = int(calc_N_bitween_dates_equill_years(last_date, last_visit_date, time_in_sec)/60/24)
    pred_date = db.loc[i, 'pred']
    days_pred_visit_and_last_date = int(calc_N_bitween_dates_equill_years(last_date, pred_date, time_in_sec)/60/24)
    v2.append(len(vv[vv > 0]))
    v0.append(days_after_last_visit)
    v1.append(days_pred_visit_and_last_date)
    
db['dif1'] = v0
db['dif2'] = v1
db['len'] = v2

100%|██████████| 7477/7477 [00:08<00:00, 857.58it/s]


In [27]:
db[['ID', 'dif1', 'dif2', 'len']]

,ID,dif1,dif2,len
0,1632,1,57,69
1,124509,1,285,2
2,43791,1,123,13
3,85655,1,285,4
4,26081,1,41,115
...,...,...,...,...
7472,378,625,618,3
7473,507,627,285,2
7474,392,632,285,2
7475,351,634,285,2


In [28]:
# Анализ количества дней после последнего визита
res_df1 = got_counter_res_df_speed1(db['dif1'])
print('Количество клиентов совершивших последний визит менее чем за 3 месяца от последней даты: {:d}'.format(np.sum(res_df1[res_df1['event'] <= 90]['count'])))
print('Количество клиентов совершивших последний визит более чем за 3 месяца от последней даты: {:d}'.format(np.sum(res_df1[res_df1['event'] > 90]['count'])))

Количество клиентов совершивших последний визит менее чем за 3 месяца от последней даты: 3980
Количество клиентов совершивших последний визит более чем за 3 месяца от последней даты: 3497


In [29]:
# Анализ количества дней между последней датой и предполагаемой датой визита
res_df2 = got_counter_res_df_speed1(db['dif2'])
print('Количество клиентов совершит визит в течении 3 месяцев от последней даты: {:d}'.format(np.sum(res_df2[res_df2['event'] <= 90]['count'])))
print('Количество клиентов совершит визит более чем через 3 месяца от последней даты: {:d}'.format(np.sum(res_df2[res_df2['event'] > 90]['count'])))

Количество клиентов совершит визит в течении 3 месяцев от последней даты: 2745
Количество клиентов совершит визит более чем через 3 месяца от последней даты: 4732


In [30]:
#Лояльные
print('Количество "лояльных" клиентов - совершивших последний визит менее чем за 1 месяца от последней даты: {:d}'.format(np.sum(res_df1[res_df1['event'] <= 30]['count'])))

Количество "лояльных" клиентов - совершивших последний визит менее чем за 1 месяца от последней даты: 2181


In [31]:
#Склонные к оттоку
print('Количество клиентов "Склонных к оттоку" совершивших последний визит более чем за 3 месяца от последней даты: {:d}'.format(np.sum(res_df1[res_df1['event'] <= 90][res_df1['event'] > 30]['count'])))

Количество клиентов "Склонных к оттоку" совершивших последний визит более чем за 3 месяца от последней даты: 1799


/Volumes/Data/opt/anaconda3/envs/kaggle_bird_36/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [32]:
#В оттоке
print('Количество клиентов "В оттоке" - совершивших последний визит более чем за 3 месяца от последней даты: {:d}'.format(np.sum(res_df1[res_df1['event'] > 90]['count'])))

Количество клиентов "В оттоке" - совершивших последний визит более чем за 3 месяца от последней даты: 3497


### Выводы

Из 7477 клиентов можем считать лояльными клиентами тех, чье поведение можем предсказать, 
их число примерно 5 + 8 процентов от общего числа.
20 процентов клиентов склонны к оттоку.
60 процентов клиентов не можем предсказать и считаем в оттоке.

Для анализа оттока мне понадобилось бы проанализировать стандартное поведение клиента и поведение в последнее время. 
Визиты, покупки, клики.